In [1]:
from multiclass_functions1 import *
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

cpu


In [2]:
# for random seed
import random
random_seed = 0
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
# BATCH_SIZE = 256
# LR = 1e-3
BATCH_SIZE = 32
LR = 1e-3
EPOCH = 100
criterion = nn.CrossEntropyLoss()
NoB = 10
NoC = 1
NoBat = 5
activation = "NoSigmoid" # NO는 Batch Nomalize 여부
new_model_train = False
model_type = f"{activation}{NoB}C{NoC}"

In [4]:
transform = transforms.ToTensor()
train_DS = datasets.MNIST(root = '../data', train=True, download=True, transform=transform)
test_DS = datasets.MNIST(root = '../data', train=False, download=True, transform=transform)
train_DL = torch.utils.data.DataLoader(train_DS, batch_size=BATCH_SIZE, shuffle=True)
test_DL = torch.utils.data.DataLoader(test_DS, batch_size=BATCH_SIZE, shuffle=True)

In [14]:
# bias = True 면 ReLU 도 잘 안된다. 일단 bias=False로 변경
in_channel = 1
# in_channel = 3
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        if activation == "NoSigmoid":
            self.conv_block = nn.Sequential(
                nn.Conv2d(in_channel,NoC,3, bias=False, padding=1),
                nn.Sigmoid(),
                *[i for j in range(NoB-1) for i in [nn.Conv2d(NoC,NoC,3, bias=False, padding=1), nn.Sigmoid()]]
                )
        elif activation == "BNSigmoid":
            self.conv_block = nn.Sequential(
                nn.Conv2d(in_channel,NoC,3, bias=False, padding=1),
                nn.BatchNorm2d(NoC), ## Activation 전에 놓기 = 데이터에 적용하기
                nn.Sigmoid(),
                *[i for j in range(NoB-1) for i in [nn.Conv2d(NoC,NoC,3, bias=False, padding=1), nn.BatchNorm2d(NoC), nn.Sigmoid()]]
                )
        elif activation == "NoReLU":
            self.conv_block = nn.Sequential(
                nn.Conv2d(in_channel,NoC,3, bias=False, padding=1),
                nn.ReLU(),
                *[i for j in range(NoB-1) for i in [nn.Conv2d(NoC,NoC,3, bias=False, padding=1), nn.ReLU()]]
                )
        elif activation == "BNReLU":
            self.conv_block = nn.Sequential(
                nn.Conv2d(in_channel,NoC,3, bias=False, padding=1),
                nn.BatchNorm2d(NoC),
                nn.ReLU(),
                *[i for j in range(NoB-1) for i in [nn.Conv2d(NoC,NoC,3, bias=False, padding=1), nn.BatchNorm2d(NoC), nn.ReLU()]]
                )
        # self.fc = nn.Linear(NoC*(28-2*NoB)*(28-2*NoB),10, bias=False)
        self.maxpool1 = nn.MaxPool2d(2)
        self.maxpool2 = nn.MaxPool2d(2)
        self.fc = nn.Linear(NoC*7*7,10, bias=False)
        # self.fc = nn.Linear(NoC*8*8,10, bias=False)

    def forward(self, x):
        self.befAct=[]
        for layer in self.conv_block:
            if isinstance(layer, nn.ReLU) or isinstance(layer, nn.Sigmoid): # Activation일 경우
              # isinstance(1, int)
                with torch.no_grad():
                    self.befAct += [torch.flatten(x).detach().cpu()] # befAct : Activation 통과 전 데이터의 분포를 담도록
                    # 보기에는 그냥 데이터 값 담는 것 같은데...
                    # 어떠한 구간에 몇 개의 데이터를 지니고 있는지 히스토그램으로 나타낼 것이다
                x = layer(x)
            else:
                x = layer(x)

        x = self.maxpool1(x)
        x = self.maxpool2(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fc(x)
        return x

In [15]:
model=CNN().to(DEVICE)
print(model)
x_batch, _ = next(iter(train_DL))
print(model(x_batch.to(DEVICE)).shape)

CNN(
  (conv_block): Sequential(
    (0): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): Sigmoid()
    (2): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (3): Sigmoid()
    (4): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (5): Sigmoid()
    (6): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): Sigmoid()
    (8): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (9): Sigmoid()
    (10): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): Sigmoid()
    (12): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (13): Sigmoid()
    (14): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (15): Sigmoid()
    (16): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (17): Sigmoid()
    (18): Conv2d(1, 1, 

In [5]:
def fmap_gen(model, train_DL, idx=5):
    # 각 layer를 지나갈 때마다 어떤 이미지의 형태로 변화하는지 확인하기 위함
    x_batch, _ = next(iter(train_DL))
    im_data = torch.unsqueeze(x_batch[idx], dim=0).to(DEVICE)
    feature_map=[]
    x = im_data
    model.eval()
    with torch.no_grad():
        for layer in model.conv_block:
            if isinstance(layer, nn.ReLU) or isinstance(layer, nn.Sigmoid):
                feature_map += [x.detach().squeeze().cpu()]
                # 테스트 단계에서 activateion을 들어가기 전 데이터들을 모아놓음
                x = layer(x)
            else:
                x = layer(x)
    return feature_map

def Train(model, train_DL, criterion):
    optimizer = optim.Adam(model.parameters(), lr=LR)

    loss_history=[]
    acc_history=[]
    befAct_history=[]
    fmap_history=[]

    NoT=len(train_DL.dataset) # The number of training data

    for ep in range(EPOCH):
        rloss = 0 # running loss
        rcorrect = 0 # running correct
        befAct_e=[]
        # fmap
        fmap_history += [fmap_gen(model, train_DL)]
        model.train() # train mode로 전환
        for i, (x_batch, y_batch) in enumerate(train_DL):
            x_batch = x_batch.to(DEVICE)
            y_batch = y_batch.to(DEVICE)
            # inference
            y_hat = model(x_batch)
            # before activation
            if i < NoBat:
                befAct_e += [model.befAct]
            # loss
            loss = criterion(y_hat, y_batch)
            # update
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # loss accumulation
            loss_b = loss.item() * x_batch.shape[0]
            rloss += loss_b
            # accuracy accumulation
            pred = y_hat.argmax(dim=1)
            corrects_b = torch.sum(pred == y_batch).item()
            rcorrect += corrects_b
        # print loss
        loss_e = rloss/NoT
        accuracy_e = rcorrect/NoT * 100
        loss_history += [loss_e]
        acc_history += [accuracy_e]
        befAct_history += [befAct_e]
        print(f"Epoch: {ep+1}, train loss: {round(loss_e,3)}, train accuracy: {round(accuracy_e,1)} %")
        print("-"*20)

    return loss_history, acc_history, befAct_history, fmap_history

In [17]:
if new_model_train:
    loss_history, acc_history, befAct_history, fmap_history = Train(model, train_DL, criterion)

In [18]:
if activation[2:] == "ReLU":
    act = "NoReLU"
elif activation[2:] == "Sigmoid":
    act = "NoSigmoid"
model_type = f"{act}{NoB}C{NoC}"

test_acc_No = Test(model, test_DL)

if activation[2:] == "ReLU":
    act = "BNReLU"
elif activation[2:] == "Sigmoid":
    act = "BNSigmoid"
model_type = f"{act}{NoB}C{NoC}"
test_acc_BN = Test(model, test_DL)

Test accuracy: 1135/10000 (11.3 %)
Test accuracy: 9437/10000 (94.4 %)


In [19]:
for layer in model.conv_block:
    if isinstance(layer, nn.BatchNorm2d):
        print(layer.weight.detach().cpu()) # nomalize 분산
        print(layer.bias.detach().cpu()) # nomalize 평균
        print("-"*20)

tensor([0.7034])
tensor([-0.8660])
--------------------
tensor([0.7678])
tensor([-0.1713])
--------------------
tensor([1.0240])
tensor([-0.2730])
--------------------
tensor([1.8532])
tensor([-0.7914])
--------------------
tensor([4.0370])
tensor([-2.0524])
--------------------
tensor([2.4948])
tensor([0.7477])
--------------------
tensor([2.3084])
tensor([1.2031])
--------------------
tensor([1.9797])
tensor([-2.0637])
--------------------
tensor([1.5177])
tensor([-1.8384])
--------------------
tensor([1.5358])
tensor([-1.8825])
--------------------


# 결과 확인

<img src="image/sigmoid.png" width="800">

sigmoid
- BN를 안했을 때(상단)와 했을 때 비교(하단)
- 각 배치별로 데이터 분포를 그린 것, 색상은 0을 기준으로 양수 음수 표현한 것
  - 각 분포 하나가 배치 하나의 분포를 나타낸 것
- 아래 BN을 했을 때, 더 옹기종기 모여있는 모습
- layer별로 어떤 이미지가 나오는지 확인 가능

마지막 layer만 봤을 때 분포가 오른쪽으로 오는 것으로 볼 수 있다
- 이미지를 봐도 위에는 제대로 안 나옴

<img src="image/sigmoid_NoBN_data.png" width="800">
<img src="image/sigmoid_BN_data.png" width="800">

데이터 분포들도 계속 gradient가 사라지는 것을 볼 수 있다
- 자꾸 왼쪽이나 오른쪽으로 치우쳐져서

<img src="image/RELU.png" width="800">

RELU
- BN이 있나 (하단) 없나 (상단) 별 차이가 없다

<img src="image/RELU_NoBN_data.png" width="800">
<img src="image/RELU_BN_data.png" width="800">

근데 BN의 파라미터 마저도 앞 레이어에 갈수록 vanishing gradient가 나오기 때문에 분포 자체가 활발하지 않은 것 보임